In [ ]:
import pandas as pd
import cobra
import copy
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

ListOfRandomSpecies = [
'Bacteroides_thetaiotaomicron_VPI_5482',
'Bacteroides_sp_2_2_4',
'Parabacteroides_johnsonii_DSM_18315',
'Prevotella_oralis_ATCC_33269',
'Eubacterium_eligens_ATCC_27750',
'Slackia_exigua_ATCC_700122',
'Dorea_longicatena_DSM_13814',
'Clostridium_bartlettii_DSM_16795',
'Streptococcus_sp_I_P16',
'Blautia_hydrogenotrophica_DSM_10507',
'Brevundimonas_bacteroides_DSM_4726',
'Clostridium_hylemonae_DSM_15053',
'Sutterella_wadsworthensis_3_1_45B',
'Enterobacteriaceae_bacterium_9_2_54FAA',
'Bacillus_megaterium_DSM319',
'Peptostreptococcus_stomatis_DSM_17678', # average-european-diet causes problems
'Brachybacterium_paraconglomeratum_LC44',
'Neisseria_elongata_subsp_glycolytica_ATCC_29315',
'Rothia_aeria_F0474',
'Staphylococcus_hominis_subsp_hominis_C80', 
]

betternames = [n.split('_')[0][0] + '.' + n.split('_')[1] for n in ListOfRandomSpecies]
stats = {}
types = [
    'high-fiber-diet',
    'unconstrained',
    'western-diet',
#    'average-european-diet',
 
]

def sensitivitySort(DietSensitivity):
    metnames=['']
    metsensitivity=[0]
    check=0
    for metabolite in DietSensitivity.keys():
        if check == 0:
            check+=1
            metnames[0]=metabolite
            metsensitivity[0]=DietSensitivity[metabolite]
        else:
            i=0
            while i<len(metnames):
                if DietSensitivity[metabolite] >  metsensitivity[i]:
                    metsensitivity[i+1:]=metsensitivity[i:]
                    metsensitivity[i]= DietSensitivity[metabolite]
                    metnames[i+1:]=metnames[i:]
                    metnames[i]=metabolite
                    break
                elif i==len(metnames)-1:
                    metsensitivity.append(DietSensitivity[metabolite])
                    metnames.append(metabolite)
                    break
                i+=1
    return metnames, metsensitivity

def getRelevantDictionaries(diet, species):
    model = cobra.io.read_sbml_model("./dfba/data/"+ diet +"/" + species + ".xml")
    AllExchanges = [r.id for r in model.exchanges]
    solution = model.optimize()
    igr = solution.objective_value
    DietSensitivity = {}
    originallb = {}
    for r in AllExchanges:
        if r not in DietSensitivity.keys():
            originallb[r] = model.reactions.get_by_id(r).lower_bound
            model.reactions.get_by_id(r).lower_bound = 0
            s = model.optimize()
            gr = s.objective_value
            DietSensitivity[r] = abs(igr-gr)/igr
            model.reactions.get_by_id(r).lower_bound = originallb[r] 
    return model, DietSensitivity, originallb

def getMinMetabolites(diet,species,SenThresh,Clusters=False):
    model, DietSensitivity, originallb = getRelevantDictionaries(diet, species)
    metnames, metsensitivity = sensitivitySort(DietSensitivity)
    s = model.optimize()
    gro = s.objective_value
    EssentialMetabolites = {}
    NonEssentialMetabolites = {}
    for i in range(len(metnames)-1,-1,-1):
#         s = model.optimize()
#         gri = s.objective_value
        model.reactions.get_by_id(metnames[i]).lower_bound = 0
        s = model.optimize()
        gr = s.objective_value
        Sens = abs(gro-gr)/gro
        if Sens > SenThresh: #if essential
            EssentialMetabolites[metnames[i]]=Sens
            model.reactions.get_by_id(metnames[i]).lower_bound = originallb[metnames[i]]
        elif originallb[metnames[i]] < 0:
            NonEssentialMetabolites[metnames[i]]=Sens
    if Clusters== False:
        return EssentialMetabolites, NonEssentialMetabolites
    else:
        return  model, DietSensitivity, originallb, EssentialMetabolites, NonEssentialMetabolites

def defineClusters(diet,species,SenThresh):
#     start=time.clock()
#     print('Clustering Now...'
    model, DietSensitivity, originallb, EssentialMetabolites, NonEssentialMetabolites = getMinMetabolites(diet,species,SenThresh,True)
    newclocktime = time.clock()
#     print('Time for MinMet =' + str(newclocktime-start))
    print(species + ' has ' + str(len(EssentialMetabolites)) + ' essential metabolites')
    print('\tand ' + str(len(NonEssentialMetabolites)) + ' non-essential metabolites')
    s = model.optimize()
    gro = s.objective_value
    Clusters = {}
    for cluster in EssentialMetabolites:
        ClusterStorage = [cluster]
        model.reactions.get_by_id(cluster).lower_bound = 0
        s = model.optimize()
        kogr = s.objective_value
        for metabolite in NonEssentialMetabolites:
            model.reactions.get_by_id(metabolite).lower_bound = originallb[metabolite]        
            s = model.optimize()
            metgr = s.objective_value
            #print('metabolite on '+ cluster + ' helps by '+str((metgr-kogr)/(gro-kogr)) )
            if (metgr-kogr)/(gro-kogr) >= 0.50:
                ClusterStorage.append(metabolite)
            model.reactions.get_by_id(metabolite).lower_bound = 0   
        Clusters[cluster] = ClusterStorage
        model.reactions.get_by_id(cluster).lower_bound = originallb[cluster]   

#     print('ClusterTime =' + str(time.clock()-newclocktime))
    return Clusters

# for i in range(0,10):
#     A= getMinMetabolites('average-european-diet','Bacteroides_thetaiotaomicron_VPI_5482',0.75)





In [ ]:
for sp in ListOfRandomSpecies:
    if 'Peptostreptococcus' in sp:
        continue
    else:
        Clusters = defineClusters('average-european-diet',sp,0.75)
        anything=0
        for A in Clusters.keys():
            if len(Clusters[A]) > 1:
                anything += 1
                print('\tCluster ' + str(anything) + ' = ' + str(Clusters[A]))
        if anything==0:
            print('\tand no clusters have size > 1')

In [ ]:
# import pandas as pd
# import cobra
# import copy
# import matplotlib.pyplot as plt
# from tqdm import tqdm


# ListOfRandomSpecies = [
# 'Bacteroides_thetaiotaomicron_VPI_5482',
# 'Bacteroides_sp_2_2_4',
# 'Parabacteroides_johnsonii_DSM_18315',
# 'Prevotella_oralis_ATCC_33269',
# 'Eubacterium_eligens_ATCC_27750',
# 'Slackia_exigua_ATCC_700122',
# 'Dorea_longicatena_DSM_13814',
# 'Clostridium_bartlettii_DSM_16795',
# 'Streptococcus_sp_I_P16',
# 'Blautia_hydrogenotrophica_DSM_10507',
# 'Brevundimonas_bacteroides_DSM_4726',
# 'Clostridium_hylemonae_DSM_15053',
# 'Sutterella_wadsworthensis_3_1_45B',
# 'Enterobacteriaceae_bacterium_9_2_54FAA',
# 'Bacillus_megaterium_DSM319',
# 'Peptostreptococcus_stomatis_DSM_17678', # average-european-diet causes problems
# 'Brachybacterium_paraconglomeratum_LC44',
# 'Neisseria_elongata_subsp_glycolytica_ATCC_29315',
# 'Rothia_aeria_F0474',
# 'Staphylococcus_hominis_subsp_hominis_C80', 
# ]

# betternames = [n.split('_')[0][0] + '.' + n.split('_')[1] for n in ListOfRandomSpecies]
# stats = {}
# types = [
#     'high-fiber-diet',
#     'unconstrained',
#     'western-diet',
# #    'average-european-diet',
 
# ]
# for t in types:
#     print(t)
#     for sp in ListOfRandomSpecies:
#         if 'Peptostreptococcus' in sp and t=='average-european-diet':
#             continue
#         else:
#             if sp not in stats.keys():
#                 stats[sp] = {}
#             print(sp)
    
#             model = cobra.io.read_sbml_model("./dfba/data/"+ t +"/" + sp + ".xml")
#             AllExchanges = [r.id for r in model.exchanges]
#             solution = model.optimize()
#             igr = solution.objective_value
#             print("\tInitial growth rate:" + str(igr))
#             DietSensitivity = {}
            
#             count = 0
#             for r in AllExchanges:
#                 if r not in DietSensitivity.keys():
#                     originallb = model.reactions.get_by_id(r).lower_bound
#                     model.reactions.get_by_id(r).lower_bound = 0.0
#                     s = model.optimize()
#                     gr = s.objective_value
#                     if abs(igr-gr)/igr > 0.1:
#                         DietSensitivity[r] = abs(igr-gr)/igr
#                         model.reactions.get_by_id(r).lower_bound = originallb
#                 count += 1
    
#             solution = model.optimize()
            
#             print("\tWith minimal influxes: " + str(solution.objective_value))
#             print("\tThe minimal set contains " + str(len(DietSensitivity.keys())) + " metabolites")
    
#             stats[sp][t]=len(DietSensitivity.keys())
            
# for s in stats:
#     print(s + '\t' + str(stats[s][types[0]]) + '\t' +  str(stats[s][types[1]]) + '\t' +  str(stats[s][types[2]]))

# pos = -0.2
# for t in types:
#     plotthis = {sp:stats[sp][t] for sp in stats.keys()}
#     plt.bar([r + pos for r in range(len(plotthis))], plotthis.values(),width=0.15, align='center', label=t)
#     pos +=0.2
# plt.xticks(range(len(plotthis)), betternames,rotation=30)
# plt.legend()
# plt.show()
    
 